In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at010-agent-chat"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='f721396f-bf18-4115-977a-5889514d2623', created_at=datetime.datetime(2025, 8, 19, 1, 5, 42, 161770, tzinfo=TzInfo(UTC)), name='a44feddb-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 8, 19, 1, 5, 42, 161770, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='38841961-f566-43c5-8eaf-8a8bd2f0d595', agent_id='a1abb90e-c673-4448-a4e2-841170568840', sequence_id=912, task_id='f721396f-bf18-4115-977a-5889514d2623', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 8, 19, 1, 5, 42, 197795, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=60,
)

╭───────────────────────── USER [08/19/2025 01:05:42] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/19/2025 01:05:47] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Searching for AI news                                                        │
│                                                                              │
│ The user wants the latest news about AI and AI startups, so I'll need to     │
│ pull recent information since my last knowledge update was in June 2024. I   │
│ plan to use the web_search tool to fetch current news. My queries should     │
│ include broad terms, like "AI news August 2025" and "AI startups funding     │
│ August 2025," to gather a comprehensive view. It seems best to combine all   │
│ these aspects into one search for efficient results. I’m ready to get        │
│ started!                                                                     │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/19/2025 01:06:11] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Summarizing AI news                                                          │
│                                                                              │
│ The web search provided a summary up to August 19, 2025, which is great! I   │
│ need to craft a concise overview of the latest AI and startup news, covering │
│ key funding rounds like Thinking Machines' notable valuation of $10B after a │
│ $2B raise, acquisitions, and regulatory moves. I'll note trends in agentic   │
│ AI and enterprise adoption while providing citations for credibility. I      │
│ should ask the user if they want to explore any specific topics in more      │
│ detail.                                                                      │
│                                                                              │
│ Preparing AI news summary                                                    │
│                                                                              │
│ I should include a brief explanation of what the latest updates mean and     │
│ highlight what to watch for, like upcoming regulatory decisions, Nvidia      │
│ market movements, and major funding rounds. I can offer to fetch live        │
│ updates if the user is interested. I'll organize the information into 8-12   │
│ bullet points, along with 4 items under "what to watch." It’s important to   │
│ verify the links from the web search to ensure they are accurate and will    │
│ lead to reliable sources.                                                    │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/19/2025 01:06:16] ─────────────────────────╮
│ Here’s a concise roundup of the latest AI and AI-startup news (as of Aug 19, │
│ 2025), with why it matters and what to watch next.                           │
│                                                                              │
│ Top headlines                                                                │
│                                                                              │
│  • Big funding & unicorn moves                                               │
│     • Thinking Machines Lab (Mira Murati): reportedly raised ~$2B and        │
│       reached a ~$10B valuation; focused on agentic / scaled AI              │
│       infrastructure. (coverage: Last Week in AI)                            │
│       Source:                                                                │
│       https://medium.com/last-week-in-ai/last-week-in-ai-august-17-2025-87d1 │
│       a35da411                                                               │
│     • Several early-stage raises reported: Rifa AI (conversational voice for │
│       enterprise) seed round, Bhindi AI (personal/business assistant)        │
│       pre-seed, Reliant AI (life-sciences analytics) launched product after  │
│       an $11.3M seed. These illustrate continued investor interest in        │
│       verticalized generative-AI startups.                                   │
│       Source:                                                                │
│       https://medium.com/last-week-in-ai/last-week-in-ai-august-17-2025-87d1 │
│       a35da411 and https://dataphoenix.info/this-week-in-ai-august-19-25/    │
│  • Acquisitions / M&A                                                        │
│     • Nvidia closed its acquisition of Run:ai (reported ~$700M), clearing    │
│       regulatory scrutiny — another step in Nvidia’s push to own more of the │
│       AI infrastructure stack.                                               │
│       Source:                                                                │
│       https://www.reuters.com/markets/deals/nvidia-closes-700-mln-runai-acqu │
│       isition-after-regulatory-hurdles-2024-12-30/                           │
│  • Talent & “acqui-hire” concerns                                            │
│     • High-profile hires from startups into Big Tech (e.g., Microsoft hiring │
│       talent from Inflection) have reignited debate about whether talent     │
│       moves and “acqui-hires” reduce competition and hurt startup            │
│       ecosystems. FT coverage explores regulatory & VC impacts.              │
│       Source:                                                                │
│       https://www.ft.com/content/4fd00a0e-3d5c-4e73-90a4-46abe5b5d40f        │
│  • Regulation / antitrust scrutiny                                           │
│     • The FTC opened inquiries into partnerships between major cloud/tech    │
│       providers and leading AI startups (questions about privileged access,  │
│       competition).                                                          │
│       Source: https://apnews.com/article/67feef411ef311f0be543f546ef34b3d    │
│     • European measures: reports of proposals to ease some compliance        │
│       burdens for early-stage AI startups (temporary relaxations to foster   │
│       innovation) — signaling regulators are trying to balance safety and    │
│       growth. (summary coverage)                                             │
│       Source:                                                                │
│       https://thecaptable.co/eu-eases-ai-startup-regulations-to-boost-innova │
│       tion/                                                                  │
│                                                                              │
│ Market & trend notes                                                         │
│                              

╭──────────────────────── AGENT [08/19/2025 01:05:51] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Searching for AI news                                                        │
│                                                                              │
│ The user wants the latest news about AI and AI startups, so I'll need to     │
│ pull recent information since my last knowledge update was in June 2024. I   │
│ plan to use the web_search tool to fetch current news. My queries should     │
│ include broad terms, like "AI news August 2025" and "AI startups funding     │
│ August 2025," to gather a comprehensive view. It seems best to combine all   │
│ these aspects into one search for efficient results. I’m ready to get        │
│ started!                                                                     │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/19/2025 01:06:25] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Summarizing AI news                                                          │
│                                                                              │
│ The web search provided a summary up to August 19, 2025, which is great! I   │
│ need to craft a concise overview of the latest AI and startup news, covering │
│ key funding rounds like Thinking Machines' notable valuation of $10B after a │
│ $2B raise, acquisitions, and regulatory moves. I'll note trends in agentic   │
│ AI and enterprise adoption while providing citations for credibility. I      │
│ should ask the user if they want to explore any specific topics in more      │
│ detail.                                                                      │
│                                                                              │
│ Preparing AI news summary                                                    │
│                                                                              │
│ I should include a brief explanation of what the latest updates mean and     │
│ highlight what to watch for, like upcoming regulatory decisions, Nvidia      │
│ market movements, and major funding rounds. I can offer to fetch live        │
│ updates if the user is interested. I'll organize the information into 8-12   │
│ bullet points, along with 4 items under "what to watch." It’s important to   │
│ verify the links from the web search to ensure they are accurate and will    │
│ lead to reliable sources.                                                    │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 60 seconds - returning collected messages
